In [486]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [487]:
# Creating training dataframe
df_train=pd.read_csv('../input/imbeside/data/train.csv',names=['image_name','smile_type'])
df_train.head()

In [488]:
# Exploring training dataframe
print('Shape of training dataframe :',df_train.shape)

In [489]:
# Checking info of training dataframe
df_train.info()

In [490]:
# Checking for null values
df_train.isnull().sum()

In [491]:
# Checking unique classes in target variable 
df_train.smile_type.unique()

In [492]:
# values count in target variable 
df_train['smile_type'].value_counts()

In [493]:
# Visualizing above values using bar plot
df_train['smile_type'].value_counts().plot(kind='bar')
plt.title('Barplot for training data')

In [494]:
# Creating testing dataframe
df_test=pd.read_csv('../input/imbeside/data/test.csv',names=['image_name','smile_type'])
df_test.head()

In [495]:
# Exploring testing dataframe
print('Shape of training dataframe :',df_test.shape)

In [496]:
# Checking info of testing dataframe
df_test.info()

In [497]:
# Checking for null values
df_test.isnull().sum()

In [498]:
# Checking unique classes in target variable 
df_test.smile_type.unique()

In [499]:
# By seeing the path of an image file we can say that the formate of file is of '.jpg' type
# adding '.jpg' at the end of every image_name in dataframe
# making a copy of dataframe so that original data will not change
train_df=df_train.copy()
def add_jpg(row):
  row=row+'.jpg'
 
  return row

train_df['image_name']=train_df['image_name'].apply(add_jpg)
#train_df['smile_type']=train_df['smile_type'].replace(
#    ['NOT smile', 'positive smile', 'negative smile'],[0,1,2])
train_df.head()

In [500]:
# doing same for test data
test_df=df_test.copy()
test_df['image_name']=test_df['image_name'].apply(add_jpg)

#test_df['smile_type']=test_df['smile_type'].replace(
 #   ['NOT smile', 'positive smile', 'negative smile'],['0','1','2'])
test_df.head()

In [501]:
# Checking on size of images
import cv2
plt.figure(figsize=(10,10))
plt.subplot(131)
img=cv2.imread('../input/imbeside/data/happy_images/happy_images/05684c70-113d-4894-920e-140edf323528.jpg')
plt.imshow(img)


plt.subplot(132)
img=cv2.imread('../input/imbeside/data/happy_images/happy_images/1ba6784e-3684-4f76-81b7-504610a497e1.jpg')
plt.imshow(img)


plt.subplot(133)
img=cv2.imread('../input/imbeside/data/happy_images/happy_images/0403faa2-2d45-4779-b40a-794c220d3855.jpg')
plt.imshow(img)

#### From aboves images we can see that there are different different pixel images but highest pixel image is 250x250
#### So we are taking our target_size=(250,250,3) in image data generator

In [502]:
# Creating training and testing imagedatagenerator to feed the network
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Applying some dataaugmentation and splitting train.csv data to training and validation
datagen = ImageDataGenerator(rescale=1/255,horizontal_flip=True,validation_split=0.15) 
dir='../input/imbeside/data/happy_images/happy_images'
train_gen = datagen.flow_from_dataframe(dataframe=train_df,
                                              directory=dir,
                                              x_col="image_name",
                                              y_col="smile_type",
                                              subset='training',
                                              target_size=(250,250),
                                              batch_size=32,
                                              color_mode='rgb',
                                              suffle=True,
                                              class_mode="categorical",
                                              seed=0
                                              )

valid_gen = datagen.flow_from_dataframe(dataframe=train_df,
                                        directory=dir,
                                        x_col='image_name',
                                        y_col='smile_type',
                                        subset='validation',
                                        target_size=(250,250),
                                        batch_size=32,
                                        color_mode='rgb',
                                        suffle=True,
                                        class_mode='categorical',
                                        seed=0
                                        )

test_datagen = ImageDataGenerator(rescale=1/255)
test_gen = test_datagen.flow_from_dataframe(dataframe=test_df,directory=dir,
                                            x_col="image_name",
                                            y_col="smile_type",
                                            target_size=(250,250),
                                            batch_size=32,
                                            color_mode='rgb',
                                            suffle=False,
                                            class_mode="categorical"
                                        )

In [503]:
# Checking shape of an image 
x,y=next(train_gen)
print(x.shape) # where 32 is batch size , 250x250 pixel image , 3 no. of color channels

In [504]:
# Checking on class labels taken as indices by image data generator
train_gen.class_indices

In [505]:
# Showing different different images with their labels
labels_dict=(train_gen.class_indices)
class_labels=list(labels_dict.keys())
def show_img(img,labels):
      plt.figure(figsize=(10,10))
      for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(img[i])
        plt.title(class_labels[np.argmax(labels[i])])
        plt.axis('off')

show_img(x,y)

# Creating model without using pretrained base

In [506]:
# Creating model just using convolution and maxpooling layers
from tensorflow.keras import layers
def model_without_pretrained():
        model=keras.Sequential([

                                    layers.Conv2D(32,(3,3),activation='relu',input_shape=(250,250,3)),
                                    layers.MaxPooling2D(2,2),
                                    layers.Conv2D(64,(3,3),activation='relu'),
                                    layers.MaxPooling2D(2,2),
                                    layers.Conv2D(128,(3,3),activation='relu'),
                                    layers.MaxPooling2D(2,2),
                                    layers.Conv2D(128,(3,3),activation='relu'),
                                    layers.MaxPooling2D(2,2),
                                    layers.Conv2D(16,(3,3),activation='relu'),
                                    layers.MaxPooling2D(2,2),


                                    layers.Flatten(),
                                    #layers.BatchNormalization(),
                                    layers.Dense(512,activation='relu'),
                                    #layers.Dense(128,activation='relu'),
                                    layers.Dropout(0.2),
                                    layers.Dense(3,activation='softmax')
            ])

        # Compiling model
        model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
        
        return model

In [507]:
# creating model using function
model=model_without_pretrained()

In [508]:
# Summary of model
model.summary()

In [509]:
# Specifying model check point
checkpoint_filepath = 'model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [510]:
# Fitting data to the model for epochs=30
hist=model.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback])

In [511]:
# Creating datframe of losses and accuracies
hist=pd.DataFrame(hist.history)
hist.head()

In [512]:
# Plotting accuracy plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist[['accuracy','val_accuracy']])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracies vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [513]:
# Plotting loss plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist[['loss','val_loss']])
plt.legend(['loss','val_loss'])
plt.title('Losses vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Losses')

In [514]:
# Evaluating model on training data
print('training evaluation :',model.evaluate(train_gen))

# Evaluating model on testing data
print('testing evaluation :',model.evaluate(test_gen))

In [515]:
# Create a basic model instance
model_ckpt = model_without_pretrained()
# Loading weights from checkpoint_path
model_ckpt.load_weights(checkpoint_filepath)
# Evaluating model on testing data
print('testing evaluation :',model_ckpt.evaluate(test_gen))  # evaluation = [Loss , Accuracy]
# Prediction From Model
preds=model_ckpt.predict(test_gen)
# Checking shape of preds
print('prediction shape',preds.shape)

# Create a function for using different different pretrained bases

In [516]:
# Now we have our training and testing data 
# Building model
from tensorflow.keras import layers,regularizers

# Defining model creating function taking parameter as pretrained_base
def create_model(pretrained_base):
    model=keras.Sequential([
                            pretrained_base,

                            layers.Flatten(),
                            #layers.BatchNormalization(),
                            #layers.Dense(256,activation='relu'),
                            layers.Dense(512,activation='relu'),
                            layers.Dropout(0.5),
                            layers.Dense(3,activation='softmax')
    ])
    
    # Compiling model
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')
    
    return model

# Using InceptionResNetV2 as pretrained base

In [517]:
# Now i need to use a pretrained model name as InceptionResNetV2
inceptionresnet=tf.keras.applications.InceptionResNetV2(
                              include_top=False,
                              input_shape=(250,250,3),
                              classes=3
)

# Making trainable equals to False
inceptionresnet.trainable=False

In [518]:
# Calling create_model function for creating model
model_incres=create_model(inceptionresnet)

In [519]:
# Summary of the model
model_incres.summary()

In [520]:
# Specifying model check point
checkpoint_filepath1 = 'model_incres.h5'
model_checkpoint_callback1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath1,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [521]:
# Fitting data to model
history=model_incres.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback1])


In [522]:
# Creating datframe of losses and accuracies
hist=pd.DataFrame(history.history)
hist.head()

In [523]:
# Plotting accuracy plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist[['accuracy','val_accuracy']])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracies vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [524]:
# Plotting loss plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist[['loss','val_loss']])
plt.legend(['loss','val_loss'])
plt.title('Losses vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Losses')

In [525]:
# Evaluating model on training data
print(model_incres.evaluate(train_gen))

In [526]:
# Evaluating model on testing data
print(model_incres.evaluate(test_gen))

In [527]:
# Create a basic model instance
model_ckpt1 = create_model(inceptionresnet)

In [528]:
# Loading weights from checkpoint_path
model_ckpt1.load_weights(checkpoint_filepath1)

In [529]:
# Evaluating model on testing data
print(model_ckpt1.evaluate(test_gen))

In [530]:
# Prediction From Model
preds1=model_ckpt1.predict(test_gen)

In [531]:
# Checking shape of preds
preds1.shape

# Using InceptionV3 as pretrained base

In [532]:
# pretrained base VGG19
inc=tf.keras.applications.InceptionV3(
                            include_top=False,
                            input_shape=(250,250,3),
                            classes=3
)

inc.trainable=False

In [533]:
#  Creating model using vgg19 as pretrained base
model_inc=create_model(inc)
model_inc.summary()

In [534]:
# Specifying model check point
checkpoint_filepath2 = 'model_inc.h5'
model_checkpoint_callback2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [535]:
# Fitting model
hist2=model_inc.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback2])

In [536]:
# Creating datframe of losses and accuracies
hist2=pd.DataFrame(hist2.history)
hist2.head()

In [537]:
# Plotting accuracy plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist2[['accuracy','val_accuracy']])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracies vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [538]:
# Plotting loss plots
plt.figure(figsize=(10,10))
plt.plot(range(1,41),hist2[['loss','val_loss']])
plt.legend(['loss','val_loss'])
plt.title('Losses vs No. of Epochs')
plt.xlabel('Epochs')
plt.ylabel('Losses')

In [539]:
# Evaluating model on training data
print(model_inc.evaluate(train_gen))

In [540]:
# Evaluating model on testing data

print(model_inc.evaluate(test_gen))

In [541]:
# Create a basic model instance
model_ckpt2 = create_model(inc)
# Loading weights from checkpoint_path
model_ckpt2.load_weights(checkpoint_filepath2)
# Evaluating model on testing data
print('testing evaluation :',model_ckpt2.evaluate(test_gen))
# Prediction From Model
preds2=model_ckpt2.predict(test_gen)
# Checking shape of preds
print('prediction shape :',preds2.shape)

# Using ResNet50 as pretrained base

In [542]:
# Taking ResNet50 as pretrained base
resnet=tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(250,250,3),
    classes=3
)

In [543]:
# using create_model function
model_resnet=create_model(resnet)

In [544]:
# model summary
model_resnet.summary()

In [545]:
# Specifying model check point
checkpoint_filepath3 = 'model_resnet.h5'
model_checkpoint_callback3 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath3,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [546]:
# training model
hist3=model_resnet.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback3])

ResNet50 is best at 17 

In [547]:
# Evaluating model on training data
print(model_resnet.evaluate(train_gen))

In [548]:
# Evaluating model on training data
print(model_resnet.evaluate(test_gen))

In [549]:
# Create a basic model instance
model_ckpt3 = create_model(resnet)
# Loading weights from checkpoint_path
model_ckpt3.load_weights(checkpoint_filepath3)
# Evaluating model on testing data
print('testing evaluation :',model_ckpt3.evaluate(test_gen))
# Prediction From Model
preds3=model_ckpt3.predict(test_gen)
# Checking shape of preds
print('prediction shape :',preds3.shape)

# Using Xception as pretrained base

In [550]:
# Pretrined base
xception=keras.applications.Xception(
    include_top=False,
    input_shape=(250,250,3),
    classes=3
)

xception.trainable=False

In [551]:
# Creating model using create_model function
model_x=create_model(xception)

In [552]:
# model summary
model_x.summary()

In [553]:
# Specifying model check point
checkpoint_filepath4 = 'model_x.h5'
model_checkpoint_callback4 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath4,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [554]:
# Fitting data to model
hist_x=model_x.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback4])

In [556]:
# Create a basic model instance
model_ckpt4 = create_model(xception)
# Loading weights from checkpoint_path
model_ckpt4.load_weights(checkpoint_filepath4)
# Evaluating model on testing data
print('testing evaluation :',model_ckpt4.evaluate(test_gen))
# Prediction From Model
preds4=model_ckpt4.predict(test_gen)
# Checking shape of preds
print('prediction shape :',preds4.shape)

# Using EfficientNet as pretrained base

In [557]:
enet=tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(250,250,3),
    classes=3
)

enet.trainable=False

In [558]:
# Creating model using  create_model function
model_enet=create_model(enet)

In [559]:
# summary of model
model_enet.summary()

In [560]:
# Specifying model check point
checkpoint_filepath5 = 'model_enet.h5'
model_checkpoint_callback5 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath5,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [561]:
# Fitting data to model
hist_enet=model_enet.fit(train_gen,validation_data=valid_gen,epochs=40,callbacks=[model_checkpoint_callback5])

In [562]:
# Checking for training accuracy 
print('training evaluation  :',model_enet.evaluate(train_gen))

In [563]:
# Checking for testing accuracy
print('testing evaluation :',model_enet.evaluate(test_gen))

In [564]:
# Create a basic model instance
model_ckpt5 = create_model(enet)
# Loading weights from checkpoint_path
model_ckpt5.load_weights(checkpoint_filepath5)
# Evaluating model on testing data
print('testing evaluation :',model_ckpt5.evaluate(test_gen))
# Prediction From Model
preds5=model_ckpt5.predict(test_gen)
# Checking shape of preds
print('prediction shape :',preds5.shape)

# ResNet50 is giving better results
# Testing Accuracy = 0.806952178478241
# Testing Loss = 0.9652607440948486